#### Results and Predictions

In [12]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd

In [2]:
prediction_model = tf.keras.models.load_model("./model/prediction/prediction_model_2/")
xy_locate_model = tf.keras.models.load_model("./model/xy_locate/xy_locate_model/")

In [3]:
frame_result = []

def start_model_predict(video_file_path):
    cap = cv2.VideoCapture(video_file_path)

    # Check if video file opened successfully
    if not cap.isOpened():
        print("Error opening video file")
        return
    # Create a window to display the video and set Mouse Callback
    cv2.namedWindow("Video Player")
        # Read each frame of the video and display it to the user
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # For each frame, do the preprocessing and normalisation, predict frame in model
        im = np.array([frame])
        im = im.astype('float32')
        im /= 255

        im = tf.image.resize(images=im, size=[240, 320])
        im = tf.keras.applications.resnet50.preprocess_input(im)
        im = im.numpy()

        prediction_output = prediction_model.predict(im)[0][0]
        xy_locate_output = xy_locate_model.predict(im)[0]

        frame_result.append([prediction_output, xy_locate_output])

    # Release video capture and close CSV file and video window
    cap.release()
    cv2.destroyAllWindows()

start_model_predict("./video/sample.mp4")

1/1 [==============================] - 0s 17ms/step


#### Data Evaluation

In [11]:
frame_result[:5]

[[0.015311731, array([380.22202, 225.3765 ], dtype=float32)],
 [0.016347358, array([382.36768, 218.42761], dtype=float32)],
 [0.015290435, array([386.47278, 202.74532], dtype=float32)],
 [0.0133118685, array([395.58734, 161.57509], dtype=float32)],
 [0.017267723, array([395.4819 , 159.40817], dtype=float32)]]

In [26]:
result = []

for i in range (len(frame_result)):
    # If prediction is >= 70%, predict staff is present in frame
    # Tag is more likely to be in the predicted XY coordinates
    if frame_result[i][0] >= 0.70:
        is_staff_in_frame = 1
        prediction = frame_result[i][0]
        X_coord_tag = frame_result[i][1][0]
        Y_coord_tag = frame_result[i][1][1]
    else:
        is_staff_in_frame = 0
        prediction = frame_result[i][0]
        X_coord_tag = None
        Y_coord_tag = None

    result.append([is_staff_in_frame, prediction, X_coord_tag, Y_coord_tag])

In [27]:
df_result = pd.DataFrame(data=result, columns=['is_staff_in_frame', 'prediction', 'X_coord_tag', 'Y_coord_tag'])
df_result.to_csv('./results.csv', index=False)
df_result.head()

,is_staff_in_frame,prediction,X_coord_tag,Y_coord_tag
0,0,0.015312,NaN,NaN
1,0,0.016347,NaN,NaN
2,0,0.015290,NaN,NaN
3,0,0.013312,NaN,NaN
4,0,0.017268,NaN,NaN
